In [1]:
import numpy as np
import cv2
import pysift
from matplotlib import pyplot as plt


In [2]:

%matplotlib inline
img1 = cv2.imread('image1.png', 0)           
img2 = cv2.imread('image2.png', 0) 

In [3]:
cv2.imshow('ab',img1)


In [4]:
kp1, des1 = pysift.computeKeypointsAndDescriptors(img1)
kp2, des2 = pysift.computeKeypointsAndDescriptors(img2)


In [5]:
print(kp1)
print(kp2)

[<KeyPoint 0x7fb13b16dcf0>, <KeyPoint 0x7fb13b1a26f0>, <KeyPoint 0x7fb13b1a27e0>, <KeyPoint 0x7fb13b1a2510>, <KeyPoint 0x7fb13b1a2450>, <KeyPoint 0x7fb13b1a2870>, <KeyPoint 0x7fb13b1a0540>, <KeyPoint 0x7fb13b1a0360>, <KeyPoint 0x7fb13b1a01e0>, <KeyPoint 0x7fb13b1a0570>, <KeyPoint 0x7fb13b1a0090>, <KeyPoint 0x7fb13b1a01b0>, <KeyPoint 0x7fb13b19fe40>, <KeyPoint 0x7fb13b1a10c0>, <KeyPoint 0x7fb13b1a12a0>, <KeyPoint 0x7fb13b1a13f0>, <KeyPoint 0x7fb13b19f210>, <KeyPoint 0x7fb13b1a2c00>, <KeyPoint 0x7fb13b1a1c60>, <KeyPoint 0x7fb13b1a1ea0>, <KeyPoint 0x7fb13b1a0870>, <KeyPoint 0x7fb13b19f360>, <KeyPoint 0x7fb13b1a2d80>, <KeyPoint 0x7fb13b19f330>, <KeyPoint 0x7fb13b19f9c0>, <KeyPoint 0x7fb13b1a1390>, <KeyPoint 0x7fb13b1a0420>, <KeyPoint 0x7fb13b16dc90>, <KeyPoint 0x7fb13b1a25d0>, <KeyPoint 0x7fb13b16dd20>, <KeyPoint 0x7fb13b1a13c0>, <KeyPoint 0x7fb13b1a2540>, <KeyPoint 0x7fb13b1a0210>, <KeyPoint 0x7fb13b1a0480>, <KeyPoint 0x7fb13b1a2780>, <KeyPoint 0x7fb13b1a1e10>, <KeyPoint 0x7fb13b1a2030>, 

In [4]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

In [5]:
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)

In [7]:
MIN_MATCH_COUNT = 10
if len(good) > MIN_MATCH_COUNT:
    # Estimate homography between template and scene
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    M = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)[0]

    # Draw detected template in scene image
    h, w = img1.shape
    pts = np.float32([[0, 0],
                      [0, h - 1],
                      [w - 1, h - 1],
                      [w - 1, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)

    img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

    h1, w1 = img1.shape
    h2, w2 = img2.shape
    nWidth = w1 + w2
    nHeight = max(h1, h2)
    hdif = int((h2 - h1) / 2)
    newimg = np.zeros((nHeight, nWidth, 3), np.uint8)

    for i in range(3):
        newimg[hdif:hdif + h1, :w1, i] = img1
        newimg[:h2, w1:w1 + w2, i] = img2

    # Draw SIFT keypoint matches
    for m in good:
        pt1 = (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1] + hdif))
        pt2 = (int(kp2[m.trainIdx].pt[0] + w1), int(kp2[m.trainIdx].pt[1]))
        cv2.line(newimg, pt1, pt2, (255, 0, 0))

    plt.imshow(newimg)
    plt.show()
else:
    print("Not enough matches are found - %d/%d" % (len(good), MIN_MATCH_COUNT))

Not enough matches are found - 7/10
